google colab 환경에서 작동하도록 만들었습니다.     
`vgg11_bn`, `vgg13_bn`에서는 잘 돌아가는 걸 확인했고     
`pytorchfi` 라이브러리 코드의 문제로 `pip`로 설치되는게 아닌 따로 수정한 버전을 가져와서 사용합니다.    
아직 해당 라이브러리 문제는 수정중이라 다른 DNN에서는 안될거에요. `resnet18`과 `resnet50`에서는 문제가 생긴걸 확인한 상태입니다.

## 시작하기 전에

CIFAR-10 pretrained weight 받아오기

1. https://github.com/huyvnphan/PyTorch_CIFAR10 중간의 구글 드라이브 링크에서 zip 파일을 다운 (약 1기가)
2. 압축 해제 후 state_dicts 폴더를 구글 드라이브에 저장


CIFAR-10 pretrained model 받아오기
1. 아래 코드 실행


몇 가지 오류를 수정한 PytorchFI 라이브러리 받아오기
1. 아래아래 코드 실행

In [ ]:
!git clone https://github.com/huyvnphan/PyTorch_CIFAR10.git

In [ ]:
!git clone https://github.com/WaiNaat/pytorchfi.git

In [ ]:
import torch
import torchvision
import torchvision.models as models
import random
import copy
import numpy as np

from PIL import Image
from torchvision import transforms
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pytorchfi
from pytorchfi.core import FaultInjection
from pytorchfi.neuron_error_models import single_bit_flip_func

from PyTorch_CIFAR10.cifar10_models.vgg import vgg11_bn, vgg13_bn, vgg16_bn, vgg19_bn
from PyTorch_CIFAR10.cifar10_models.resnet import resnet18, resnet34, resnet50
from PyTorch_CIFAR10.cifar10_models.densenet import densenet121, densenet161, densenet169
from PyTorch_CIFAR10.cifar10_models.mobilenetv2 import mobilenet_v2
from PyTorch_CIFAR10.cifar10_models.googlenet import googlenet
from PyTorch_CIFAR10.cifar10_models.inception import inception_v3

## 설정 및 모델 불러오기

---

`model_name`, `model`: 위 셀의 `PyTorch_CIFAR10.cifar10_models` 에서 `import` 한 것들 중 하나      
`layer_type`: `['all']` 또는 `torch.nn.Modules`를 상속하는 클래스명으로 구성된 iterable   
`layer_nums`: `['all']` 또는 0 이상의 정수로 구성된 배열    
`corrupt_input_images`: `True`로 설정 시 inference 전 입력 이미지 자체에도 single bit flip 적용


In [ ]:
# 실험 환경 설정
model_name = "vgg11_bn"
model = vgg11_bn()

seed = 1234

batch_size = 256
img_size = 32
channels = 3

use_gpu = torch.cuda.is_available()

corrupt_input_images = True
quant_bits = 32
layer_type = ['all']
layer_nums = ['all']

save_dir = 'vgg11_bn_all.txt'

print(use_gpu)

In [ ]:
random.seed(seed)
torch.manual_seed(seed)

In [ ]:
class add_input_layer(torch.nn.Module):

    def __init__(self, model, *args):
        super().__init__(*args)
        self.input_layer = torch.nn.Identity()
        self.model = model

    def forward(self, x):
        input = self.input_layer(x)
        output = self.model(input)
        return output

In [ ]:
# 모델 설정
path = f"/content/drive/My Drive/state_dicts/{model_name}.pt"
model.load_state_dict(torch.load(path))

if corrupt_input_images:
    model = add_input_layer(model)

if use_gpu: model.to(device='cuda')

print(model)

## 데이터 전처리

In [ ]:
# Transform statics from https://github.com/huyvnphan/PyTorch_CIFAR10/blob/master/data.py
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize([0.4914, 0.4822, 0.4465], (0.2471, 0.2435, 0.2616))
    ]
)

data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
dataset = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)

## Main

In [ ]:
# single bit flip을 일으킬 모델 만들기
base_fi_model = single_bit_flip_func(
    model = copy.deepcopy(model),
    batch_size = batch_size, 
    input_shape = [channels, img_size, img_size], 
    use_gpu = use_gpu,
    bits = quant_bits,
    layer_types = layer_type
)

print(base_fi_model.print_pytorchfi_layer_summary())

In [ ]:
# single bit flip을 수행할 layer 번호 정리
if 'all' in layer_nums:
    layer_nums = range(base_fi_model.get_total_layers())
    print(f'Total {base_fi_model.get_total_layers()} layers')
else:
    layer_nums.sort()
    while layer_nums and layer_nums[-1] >= base_fi_model.get_total_layers():
        layer_nums.pop()
    print(f'Total {len(layer_nums)} layers')

In [ ]:
# 실험 진행
results = []

for layer_num in layer_nums:

    orig_correct_cnt = 0
    orig_corrupt_diff_cnt = 0

    for images, labels in tqdm(dataset):

        if use_gpu:
            images = images.to(device='cuda')

        # 원본에 inference 진행
        model.eval()
        with torch.no_grad():
            orig_output = model(images)

        # single bit flip 위치 지정
        layer_num_list = []
        dim1 = []
        dim2 = []
        dim3 = []

        for _ in range(batch_size):
            layer, C, H, W = pytorchfi.neuron_error_models.random_neuron_location(base_fi_model, layer=layer_num)

            layer_num_list.append(layer)
            dim1.append(C)
            dim2.append(H)
            dim3.append(W)

        # corrupted model 만들기
        corrupted_model = base_fi_model.declare_neuron_fault_injection(
            batch = [i for i in range(batch_size)],
            layer_num = layer_num_list,
            dim1 = dim1,
            dim2 = dim2,
            dim3 = dim3,
            function = base_fi_model.single_bit_flip_signed_across_batch
        )

        # corrupted model에 inference 진행
        corrupted_model.eval()
        with torch.no_grad():
            corrupted_output = corrupted_model(images)

        # 결과 정리
        original_output = torch.argmax(orig_output, dim=1).cpu().numpy()
        corrupted_output = torch.argmax(corrupted_output, dim=1).cpu().numpy()
        labels = labels.numpy()

        # 결과 비교: 원본이 정답을 맞춘 경우 중 망가진 모델이 틀린 경우를 셈
        for i in range(batch_size):

            if labels[i] == original_output[i]:
                orig_correct_cnt += 1

                if original_output[i] != corrupted_output[i]:
                    orig_corrupt_diff_cnt += 1

    # 결과 저장
    result = f'Layer #{layer_num}: {orig_corrupt_diff_cnt} / {orig_correct_cnt} = {orig_corrupt_diff_cnt / orig_correct_cnt * 100:.4f}%'
    print(result, end='\n\n')
    results.append(result)

In [ ]:
for result in results:
    print(result)

## 결과 파일 저장

In [ ]:
f = open('/content/drive/MyDrive/' + save_dir, 'w')

f.write(base_fi_model.print_pytorchfi_layer_summary())
f.write(f'\n\n===== Result =====\nQuantization bits: {quant_bits}\n')
for result in results:
    f.write(result + '\n')

f.close()